# Coding copilot with Autogen

In [1]:
## import required libraries
import os
from autogen import AssistantAgent, ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor
from dotenv import load_dotenv


load_dotenv() # load environment variables from .env file

True

In [2]:
## llm configuration
llm_config ={
                "model": "gpt-4o",
                "api_key": os.getenv("OPENAI_API_KEY")
            }

## Custom functions for stock analysis

In [ ]:
## function to get stock data
def get_stock_prices(
    stock_symbols,
    start_date,
    end_date
):
    """
    Fetch stock closing prices for given stock symbols between start and end dates.

    Args:
        stock_symbols: Stock symbol(s) to fetch prices for. Can be a string or a list of strings.
        start_date: The start date in 'YYYY-MM-DD' format.
        end_date: The end date in 'YYYY-MM-DD' format.

    Returns:
        DataFrame: Stock closing prices indexed by date, with one column per stock symbol.
    """
    import yfinance
    from pandas import DataFrame
    
    try:
        stock_data = yfinance.download(stock_symbols, start=start_date, end=end_date)
        return stock_data["Close"]
    except Exception as e:
        print(f"Error fetching stock data: {str(e)}")
        return DataFrame()

In [ ]:
## plot the stock prices
def plot_stock_prices(stock_prices: DataFrame, filename: str) -> None:
    """
    Plot stock prices and save the figure to a file.

    Args:
        stock_prices (DataFrame): Stock prices indexed by date, with columns for each stock.
        filename (str): Name of the file to save the plot.
    """
    import matplotlib.pyplot as plt
    from pandas import DataFrame
    
    plt.figure(figsize=(10, 5))
    stock_prices.plot(ax=plt.gca(), grid=True)
    plt.title("Stock Prices")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend(loc='best')
    plt.savefig(filename)
    plt.close()

## A new executor with custom functions

In [ ]:
executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir="coding",
    functions=[get_stock_prices, plot_stock_prices],
)

In [ ]:
code_writer_agent_system_message += executor.format_functions_for_prompt()
print(code_writer_agent_system_message)

## Create agents with updated system message

In [ ]:
code_writer_agent = ConversableAgent(
    name="code_writer_agent",
    system_message=code_writer_agent_system_message,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [ ]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=f"Today is {today}."
    "Download the stock prices YTD for NVDA and TSLA and create"
    "a plot. Make sure the code is in markdown code block and "
    "save the figure to a file stock_prices_YTD_plot.png.",
)

## Plot the results

In [ ]:
Image(os.path.join("coding", "stock_prices_YTD_plot.png"))